In [2]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
matplotlib.style.use('fivethirtyeight')

In [3]:
def reindex_that_stuff(df):
    columns = list(df.columns)
    columns.remove("NACCUDSD")
    columns.append("NACCUDSD")
    return df.reindex(columns=columns)

CSF = pd.read_csv("CSF dataset for pipeline - next steps are tt split, dropna in test, impute in train.csv", index_col="Unnamed: 0")
MRI = pd.read_csv("MRI Dataset for pipeline - next steps are tt split.csv", index_col="Unnamed: 0")
UDS = pd.read_csv("UDS dataset for pipeline - next steps are tt split.csv", index_col="Unnamed: 0")
UDS_MRI = pd.read_csv("UDS_MRI dataset for pipeline - next steps are tt split.csv", index_col="Unnamed: 0")

UDS_MRI = reindex_that_stuff(UDS_MRI)
CSF = reindex_that_stuff(CSF)
MRI = reindex_that_stuff(MRI)
UDS = reindex_that_stuff(UDS)

# UDS_MRI = pd.read_csv("C:/Users/tminh/Downloads/results of yueqi code/DATA/Final 6.27 - " + "UDS_MRI_TRAIN" + ".csv", index_col="Unnamed: 0").append(pd.read_csv("C:/Users/tminh/Downloads/results of yueqi code/DATA/Final 6.27 - " + "UDS_MRI_TEST" + ".csv", index_col="Unnamed: 0"))

In [4]:
def binarize(df):
  new_df = df.copy()
  new_df['NACCUDSD'] = new_df['NACCUDSD'].replace(1, 0).replace([2, 3, 4], 1)
  return new_df

# Data Splitters

In [5]:
def split_csf(seed):
    from sklearn.model_selection import train_test_split

    shuffled_df = CSF.sample(frac=1, random_state=seed)
    train_df, test_df = train_test_split(shuffled_df, test_size=0.25, random_state=seed)

    print("CSF Train set shape:", train_df.shape)
    print("CSF Test set shape:", test_df.shape)
    
    # SAVING
    test_df_name = 'Mid-pipeline/CSF_TEST-' + str(seed) + '.csv'
    test_df = test_df.dropna()
    test_df.to_csv(test_df_name)
    
    from sklearn.impute import KNNImputer
    
    CSF_imputed = train_df[['CSFABETA', 'CSFTTAU', 'CSFPTAU']] # w/out NACCUDSD
    imputer = KNNImputer(n_neighbors=5)
    CSF_imputed = imputer.fit_transform(CSF_imputed)

    CSF_imputed = pd.DataFrame(CSF_imputed, columns = ['CSFABETA', 'CSFTTAU', 'CSFPTAU'])
    CSF_imputed['NACCUDSD'] = train_df.reset_index()['NACCUDSD']
    
    # SAVING    
    train_df_name = 'Mid-pipeline/CSF_TRAIN-' + str(seed) + '.csv'
    CSF_imputed.to_csv(train_df_name)
    
    return CSF_imputed, train_df_name, test_df, test_df_name

split_csf(42)

CSF Train set shape: (1251, 4)
CSF Test set shape: (418, 4)


(      CSFABETA  CSFTTAU  CSFPTAU  NACCUDSD
 0       447.00   424.00    58.30         1
 1       922.45   263.11    59.52         1
 2       162.65   906.65   106.15         4
 3       755.00  1095.00   138.30         4
 4      1023.00   205.00    35.30         1
 ...        ...      ...      ...       ...
 1246    899.00   251.00    32.90         1
 1247    618.00   534.00    74.90         3
 1248    299.27    32.90    12.91         1
 1249    779.00   518.00    68.50         1
 1250    591.00   717.00   113.90         1
 
 [1251 rows x 4 columns],
 'Mid-pipeline/CSF_TRAIN-42.csv',
       CSFABETA  CSFTTAU  CSFPTAU  NACCUDSD
 1770    739.00   171.00    23.50         1
 1101    493.72    39.43    28.58         1
 697     222.77    84.77    43.27         4
 2731    680.00   176.00    32.00         1
 2951   1075.00   331.55    64.52         1
 ...        ...      ...      ...       ...
 2016    463.00   460.00    61.50         4
 2776    395.00   195.00    22.00         1
 631     162.0

In [6]:
def split_mri(seed):
    from sklearn.model_selection import train_test_split
    
    shuffled_df = MRI.sample(frac=1, random_state=seed)
    train_df, test_df = train_test_split(shuffled_df, test_size=0.25, random_state=seed)

    # SAVING
    test_df_name = 'Mid-pipeline/MRI_TEST-' + str(seed) + '.csv'
    test_df.dropna().to_csv(test_df_name)
    
    # SAVING    
    train_df_name = 'Mid-pipeline/MRI_TRAIN-' + str(seed) + '.csv'
    train_df.to_csv(train_df_name)
    
    
    print("Train set shape:", train_df.shape)
    print("Test set shape:", test_df.shape)
    
    return train_df, train_df_name, test_df, test_df_name

split_mri(32)

Train set shape: (1239, 156)
Test set shape: (414, 156)


(       NACCICV  NACCBRNV  NACCWMVL   CSFVOL  GRAYVOL  WHITEVOL   WMHVOL  \
 245   1309.939   979.232   412.224  324.267  573.448   405.784   6.4401   
 1069  1348.544  1002.781   456.649  338.298  553.597   449.184   7.4650   
 1564  1536.840  1217.530   534.820  319.310  682.710   534.820   0.0000   
 368   1528.410  1073.670   460.020  439.870  628.520   445.150  14.8700   
 627   1661.180  1174.312   552.988  473.883  634.309   540.003  12.9847   
 ...        ...       ...       ...      ...      ...       ...      ...   
 296   1277.296   984.181   425.073  289.742  562.481   421.700   3.3732   
 335   1349.506   972.928   446.479  325.322  577.705   395.223  51.2560   
 1038  1672.960  1179.260   501.410  471.690  699.860   479.400  22.0100   
 189   1170.211   897.902   416.863  264.556  488.792   409.110   7.7533   
 374   1383.417   991.867   406.221  376.903  600.293   391.574  14.6470   
 
       HIPPOVOL   CEREALL  CERETISS  ...  RSUPFRM  RSUPPAR  RSUPPARM  RSUPTEM  \
 245 

In [7]:
def split_uds(seed):
    from sklearn.model_selection import train_test_split
    
    shuffled_df = UDS.sample(frac=1, random_state=seed)
    train_df, test_df = train_test_split(shuffled_df, test_size=0.25, random_state=seed)

    # SAVING
    test_df_name = 'Mid-pipeline/UDS_TEST-' + str(seed) + '.csv'
    test_df.dropna().to_csv(test_df_name)
    
    # SAVING    
    train_df_name = 'Mid-pipeline/UDS_TRAIN-' + str(seed) + '.csv'
    train_df.to_csv(train_df_name)
    
    
    print("Train set shape:", train_df.shape)
    print("Test set shape:", test_df.shape)
    
    return train_df, train_df_name, test_df, test_df_name

split_uds(32)

Train set shape: (14535, 110)
Test set shape: (4846, 110)


(       NACCAPOE  ANIMALS   VEG  TRAILA  TRAILARR  TRAILALI  TRAILB  TRAILBRR  \
 719         1.0      8.0   4.0    45.0       0.0      24.0   996.0      96.0   
 7395        2.0      5.0   1.0   997.0      97.0      97.0   997.0      97.0   
 12882       2.0     17.0  12.0    26.0       0.0      24.0   163.0       2.0   
 2768        3.0     23.0  18.0    35.0       0.0      24.0    86.0       0.0   
 15245       1.0     17.0  11.0   996.0      96.0      96.0   996.0      96.0   
 ...         ...      ...   ...     ...       ...       ...     ...       ...   
 11503       1.0     25.0  22.0    20.0       0.0      24.0    40.0       0.0   
 12266       3.0     15.0   8.0    94.0       0.0      24.0   300.0       3.0   
 3998        1.0     15.0   8.0    31.0       0.0      24.0    88.0       0.0   
 11729       1.0     12.0   7.0    35.0       0.0      24.0    77.0       0.0   
 8056        2.0     96.0  96.0   996.0      96.0      96.0   996.0      96.0   
 
        TRAILBLI  DECSUB  

In [8]:
def split_uds_mri(seed):
    from sklearn.model_selection import train_test_split
    
    shuffled_df = UDS_MRI.sample(frac=1, random_state=seed)
    train_df, test_df = train_test_split(shuffled_df, test_size=0.25, random_state=seed)

    # SAVING
    test_df_name = 'Mid-pipeline/UDS_MRI_TEST-' + str(seed) + '.csv'
    test_df.to_csv(test_df_name)
    
    # SAVING    
    train_df_name = 'Mid-pipeline/UDS_MRI_TRAIN-' + str(seed) + '.csv'
    train_df.to_csv(train_df_name)
    
    
    print("Train set shape:", train_df.shape)
    print("Test set shape:", test_df.shape)
    
    return train_df, train_df_name, test_df, test_df_name

split_uds_mri(32)

Train set shape: (1068, 265)
Test set shape: (357, 265)


(      NACCAPOE  ANIMALS   VEG  TRAILA  TRAILARR  TRAILALI  TRAILB  TRAILBRR  \
 66         2.0      9.0   3.0    32.0       1.0      24.0    91.0       1.0   
 527        2.0     19.0  18.0    30.0       0.0      24.0    86.0       1.0   
 1235       1.0     22.0  15.0    17.0       0.0      24.0    34.0       0.0   
 1098       1.0      5.0  10.0   110.0       0.0      24.0   300.0       2.0   
 188        1.0     20.0  13.0    28.0       0.0      24.0    69.0       1.0   
 ...        ...      ...   ...     ...       ...       ...     ...       ...   
 1154       2.0     31.0  23.0    16.0       0.0      24.0    35.0       0.0   
 157        3.0     21.0  22.0    18.0       0.0      24.0    46.0       0.0   
 1124       2.0     26.0  12.0    27.0       1.0      24.0    48.0       0.0   
 125        1.0     13.0  18.0    48.0       1.0      24.0   128.0       0.0   
 877        2.0     23.0  15.0    26.0       0.0      24.0   121.0       2.0   
 
       TRAILBLI  DECSUB  ...  RSUPFRM 

# Fusion Methods

## AE

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import gc

def AE(df, epochs = 10):

  # Separate the input features (explanatory variables) and the response variable
  X = df.drop('NACCUDSD', axis=1)  # Input features
  y = df['NACCUDSD']  # Response variable

  # Perform one-hot encoding on the categorical variable
  encoder = LabelEncoder()
  y_encoded = encoder.fit_transform(y)
  y_categorical = pd.get_dummies(y_encoded)

  # # Split the data into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0., random_state=42)
  y_train = y_categorical

  # Scale the input features
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X)
  # X_test_scaled = scaler.transform(X_test)

  # Create the neural network model
  input_shape=(X_train_scaled.shape[1],)
  print(input_shape)

  model = Sequential()
  model.add(Dense(64, activation='relu', input_shape = input_shape))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(4, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(input_shape[0], activation='softmax'))  # Output layer with softmax activation for categorical variables

  # Compile the model
  model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

  # Train the model
  model.fit(X_train_scaled, X_train_scaled, epochs=epochs, batch_size=16)

  return model, scaler
# PART 2: GET VALUES

def through_AE(df, model, scaler):
  print("started")

  # Separate the input features (explanatory variables) and the response variable
  X = df.drop('NACCUDSD', axis=1)  # Input features
  y = df['NACCUDSD']

  # Perform one-hot encoding on the categorical variable
  encoder = LabelEncoder()
  y_encoded = encoder.fit_transform(y)
  y_categorical = pd.get_dummies(y_encoded)

  # Scale the input features
  # scaler = StandardScaler()
  X_scaled = scaler.transform(X)


  import tensorflow as tf
  # Create a new model to get values from the last dense layer before output
  last_dense_layer_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-4].output)

  output_values = last_dense_layer_model.predict(X_scaled)
  new_df = pd.DataFrame(output_values)
  new_df['NACCUDSD'] = df.reset_index()['NACCUDSD']

  return new_df

## LASSO

In [10]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

def lasso(df, al, thresh):

  # Create the Lasso model
  lasso = Lasso(alpha=al)

  # Select the features
  selector = SelectFromModel(lasso, threshold=thresh)
  selector.fit(df.drop("NACCUDSD", axis=1), df["NACCUDSD"])

  # Get the selected features
  selected_features = selector.get_support()

  return df.columns[:-1][selected_features].tolist()

## SDNN



In [11]:
def through_SDNN_old(df, model, scaler):

  # Separate the input features (explanatory variables) and the response variable
  X = df.drop('NACCUDSD', axis=1)  # Input features
  y = df['NACCUDSD']

  # Perform one-hot encoding on the categorical variable
  encoder = LabelEncoder()
  y_encoded = encoder.fit_transform(y)
  y_categorical = pd.get_dummies(y_encoded)

  # # Split the data into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

  # Scale the input features
  # scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)

  import tensorflow as tf
  # Create a new model to get values from the last dense layer before output
  last_dense_layer_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)


    
  pickle = []
  for i in range(0, len(X_scaled)):
    # print(i)
    if i % 1000 == 0:
      print(str(i) + " / " + str(len(X_scaled)))
      that_df = pd.DataFrame(pickle)

    input_row = X_scaled[i]  # Replace with your desired input row

    # Obtain values from the last dense layer before output for the input row
    output_values = last_dense_layer_model.predict(input_row.reshape(1, -1), verbose=0)[0]
    # print(output_values)
    pickle.append(output_values)
    gc.collect()



  # print('Output values:', output_values)
  # new_df = pd.DataFrame(output_values)
  new_df = pd.DataFrame(pickle)
  new_df['NACCUDSD'] = df.reset_index()['NACCUDSD']

  return new_df

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import gc

def SDNN(df, epochs = 10):

  # Separate the input features (explanatory variables) and the response variable
  X = df.drop('NACCUDSD', axis=1)  # Input features
  y = df['NACCUDSD']  # Response variable

  # Perform one-hot encoding on the categorical variable
  encoder = LabelEncoder()
  y_encoded = encoder.fit_transform(y)
  y_categorical = pd.get_dummies(y_encoded)

  # # Split the data into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0., random_state=42)
  y_train = y_categorical

  # Scale the input features
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X)
  # X_test_scaled = scaler.transform(X_test)

  # Create the neural network model
  model = Sequential()
  model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(y_categorical.shape[1], activation='softmax'))  # Output layer with softmax activation for categorical variables

  # Compile the model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Train the model
  model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=16)

  return model, scaler
# PART 2: GET VALUES

def through_SDNN(df, model, scaler):

  # Separate the input features (explanatory variables) and the response variable
  X = df.drop('NACCUDSD', axis=1)  # Input features
  y = df['NACCUDSD']

  # Perform one-hot encoding on the categorical variable
  encoder = LabelEncoder()
  y_encoded = encoder.fit_transform(y)
  y_categorical = pd.get_dummies(y_encoded)

  # # Split the data into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

  # Scale the input features
  # scaler = StandardScaler()
  X_scaled = scaler.transform(X)

    
  import tensorflow as tf
  # Create a new model to get values from the last dense layer before output
  last_dense_layer_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)


  output_values = last_dense_layer_model.predict(X_scaled, verbose=0)

  # print('Output values:', output_values)
  new_df = pd.DataFrame(output_values)
  # new_df = pd.DataFrame(pickle)
  new_df['NACCUDSD'] = df.reset_index()['NACCUDSD']

  return new_df

## PCA

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def pca(train_df, test_df, var = 0.5):


  # Load and preprocess the training dataset
  train_data = train_df.reset_index().copy()
  train_features = train_data.drop('NACCUDSD', axis=1)  # Exclude the response variable

  # Standardize the training features by removing the mean and scaling to unit variance
  scaler = StandardScaler()
  scaled_train_features = scaler.fit_transform(train_features)

  # Perform PCA on the training features
  pca = PCA()
  pca.fit(scaled_train_features)

  # Load and preprocess the test dataset
  test_data = test_df.reset_index().copy()
  test_features = test_data.drop('NACCUDSD', axis=1)  # Exclude the response variable

  # Standardize the test features using the same scaler applied to the training features
  scaled_test_features = scaler.transform(test_features)

  # Apply the learned PCA model to both training and test features
  train_pca = pca.transform(scaled_train_features)
  test_pca = pca.transform(scaled_test_features)

  # Construct new DataFrames for the transformed features
  train_pca_df = pd.DataFrame(data=train_pca, columns=[f'PC{i+1}' for i in range(pca.n_components_)])
  test_pca_df = pd.DataFrame(data=test_pca, columns=[f'PC{i+1}' for i in range(pca.n_components_)])

  # Concatenate the response variable with the PCA-transformed training features
  train_pca_df = pd.concat([train_pca_df, train_data['NACCUDSD']], axis=1)
  test_pca_df = pd.concat([test_pca_df, test_data['NACCUDSD']], axis=1)

  # Print the resulting PCA-transformed datasets
  return [train_pca_df, test_pca_df]


# Classification Methods

## LogReg

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def logistic_regression(train_df, test_df, cols, give_table = False):

  train_df = binarize(train_df)
  test_df =  binarize(test_df)


  X_train = train_df[cols]  # Input features # ERROR: y is included
  y_train = train_df['NACCUDSD']   # Target variable

  X_test = test_df[cols]  # Input features # ERROR: y is included
  y_test = test_df['NACCUDSD']   # Target variable

  logreg = LogisticRegression(solver='lbfgs')
  logreg.fit(X_train, y_train)

  y_pred = logreg.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  print(accuracy)

  # if give_table:
  #       return y_pred

  return [accuracy, y_pred]

## RANDOM FOREST

In [15]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


def random_forest(train_df, test_df, cols, give_table = False):
  # Load the dataset
  # dataset = UDS_MRI_CSF[concat_cols+['NACCUDSD']]
  X_train = train_df[cols]  # Input features # ERROR: y is included
  y_train = train_df['NACCUDSD']   # Target variable

  X_test = test_df[cols]  # Input features # ERROR: y is included
  y_test = test_df['NACCUDSD']   # Target variable

  # Split the data into training and testing sets
  # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


  rf_classifier = RandomForestClassifier(n_estimators=50)

  # Fit the model to the training data
  rf_classifier.fit(X_train, y_train)

  # Make predictions on the test set
  y_pred = rf_classifier.predict(X_test)

  # Evaluate model performance
  accuracy = accuracy_score(y_test, y_pred)
  # print("Accuracy:", accuracy)
  print(accuracy)

  # if give_table:
  #       return y_pred

  # return rf_classifier
  return [accuracy, y_pred]


## Feed forward neural network

In [16]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# Assuming you have train_df and test_df DataFrames
# and the response variable is 'NACCUDSD'

def neural_network(train_df, test_df, cols, epochs = 10, give_table = False):
  # Separate features and target variable
  X_train = train_df[cols]
  y_train = train_df['NACCUDSD']-1
  X_test = test_df[cols]
  y_test = test_df['NACCUDSD']-1

  # Convert the target variable to categorical
  num_classes = len(set(y_train))
  y_train_cat = to_categorical(y_train, num_classes)
  y_test_cat = to_categorical(y_test, num_classes)

  # Standardize the features
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  # Create the model
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))

  # Compile the model
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # Train the model
  model.fit(X_train_scaled, y_train_cat, epochs=epochs, batch_size=32, verbose=1)

  # Evaluate the model
  loss, accuracy = model.evaluate(X_test_scaled, y_test_cat, verbose=0)

  print("Test loss:", loss)
  print("Test accuracy:", accuracy)
    
  if give_table:
        preds = model.predict(X_test_scaled)
        return np.argmax(preds, axis=1)

  # return model
  preds = model.predict(X_test_scaled)
  
  return [accuracy, np.argmax(preds, axis=1)]

In [17]:
input_mri_feats = ['NACCICV', 'NACCBRNV',
       'NACCWMVL', 'CSFVOL', 'GRAYVOL', 'WHITEVOL', 'WMHVOL', 'HIPPOVOL',
       'CEREALL', 'CERETISS', 'CERECSF', 'CEREGR', 'CEREWH', 'LHIPPO',
       'RHIPPO', 'LLATVENT', 'RLATVENT', 'LATVENT', 'THIRVENT', 'LFRCORT',
       'RFRCORT', 'FRCORT', 'LOCCORT', 'ROCCORT', 'OCCCORT', 'LPARCORT',
       'RPARCORT', 'PARCORT', 'LTEMPCOR', 'RTEMPCOR', 'TEMPCOR', 'LCAC',
       'LCACM', 'LCMF', 'LCMFM', 'LCUN', 'LCUNM', 'LENT', 'LENTM', 'LFUS',
       'LFUSM', 'LINFPAR', 'LINFPARM', 'LINFTEMP', 'LINFTEMM', 'LINSULA',
       'LINSULAM', 'LISTHC', 'LISTHCM', 'LLATOCC', 'LLATOCCM', 'LLATORBF',
       'LLATORBM', 'LLING', 'LLINGM', 'LMEDORBF', 'LMEDORBM', 'LMIDTEMP',
       'LMIDTEMM', 'LPARCEN', 'LPARCENM', 'LPARHIP', 'LPARHIPM',
       'LPARSOP', 'LPARSOPM', 'LPARORB', 'LPARORBM', 'LPARTRI',
       'LPARTRIM', 'LPERCAL', 'LPERCALM', 'LPOSCEN', 'LPOSCENM',
       'LPOSCIN', 'LPOSCINM', 'LPRECEN', 'LPRECENM', 'LPRECUN',
       'LPRECUNM', 'LROSANC', 'LROSANCM', 'LROSMF', 'LROSMFM', 'LSUPFR',
       'LSUPFRM', 'LSUPPAR', 'LSUPPARM', 'LSUPTEM', 'LSUPTEMM', 'LSUPMAR',
       'LSUPMARM', 'LTRTEM', 'LTRTEMM', 'RCAC', 'RCACM', 'RCMF', 'RCMFM',
       'RCUN', 'RCUNM', 'RENT', 'RENTM', 'RFUS', 'RFUSM', 'RINFPAR',
       'RINFPARM', 'RINFTEMP', 'RINFTEMM', 'RINSULA', 'RINSULAM',
       'RISTHC', 'RISTHCM', 'RLATOCC', 'RLATOCCM', 'RLATORBF', 'RLATORBM',
       'RLING', 'RLINGM', 'RMEDORBF', 'RMEDORBM', 'RMIDTEMP', 'RMIDTEMM',
       'RPARCEN', 'RPARCENM', 'RPARHIP', 'RPARHIPM', 'RPARSOP',
       'RPARSOPM', 'RPARORB', 'RPARORBM', 'RPARTRI', 'RPARTRIM',
       'RPERCAL', 'RPERCALM', 'RPOSCEN', 'RPOSCENM', 'RPOSCIN',
       'RPOSCINM', 'RPRECEN', 'RPRECENM', 'RPRECUN', 'RPRECUNM',
       'RROSANC', 'RROSANCM', 'RROSMF', 'RROSMFM', 'RSUPFR', 'RSUPFRM',
       'RSUPPAR', 'RSUPPARM', 'RSUPTEM', 'RSUPTEMM', 'RSUPMAR',
       'RSUPMARM', 'RTRTEM', 'RTRTEMM']

input_demo_feats = ['SEX', 'HISPANIC', 'HISPOR', 'RACE', 'RACEX',
       'PRIMLANG', 'EDUC', 'MARISTAT', 'NACCLIVS', 'INDEPEND', 'RESIDENC',
       'NACCNIHR','NACCAGE']
input_copart_feats = ['INEDUC', 'INRELTO', 'INKNOWN', 'INLIVWTH', 'INVISITS', 'INCALLS',
       'INRELY']
input_fam_hist_feats = ['NACCFAM', 'NACCMOM', 'NACCDAD', 'NACCAM', 'NACCAMX',
       'NACCAMS', 'NACCAMSX', 'NACCFM', 'NACCFMX', 'NACCFMS', 'NACCFMSX',
       'NACCOM', 'NACCOMX', 'NACCOMS', 'NACCOMSX', 'NACCFADM', 'NACCFFTD']
input_patient_hist_feats = ['ANYMEDS', 'TOBAC30', 'TOBAC100', 'SMOKYRS', 'PACKSPER',
       'QUITSMOK', 'CVHATT', 'CVAFIB', 'CVANGIO', 'CVBYPASS', 'CVPACDEF',
       'CVPACE', 'CVCHF', 'CVOTHR', 'CBSTROKE', 'NACCSTYR', 'CBTIA',
       'NACCTIYR', 'SEIZURES', 'NACCTBI', 'TBI', 'TBIBRIEF', 'TRAUMBRF',
       'TBIEXTEN', 'TRAUMEXT', 'TBIWOLOS', 'TRAUMCHR', 'TBIYEAR',
       'NCOTHR', 'DIABETES', 'DIABTYPE', 'HYPERTEN', 'HYPERCHO', 'B12DEF',
       'THYROID', 'ARTHRIT', 'ARTHTYPE', 'ARTHTYPX', 'ARTHUPEX',
       'ARTHLOEX', 'ARTHSPIN', 'ARTHUNK', 'INCONTU', 'INCONTF', 'APNEA',
       'RBD', 'INSOMN', 'OTHSLEEP', 'OTHSLEEX', 'ALCOHOL', 'ABUSOTHR',
       'ABUSX', 'PTSD', 'BIPOLAR', 'SCHIZ', 'DEP2YRS', 'DEPOTHR',
       'ANXIETY', 'OCD', 'NPSYDEV', 'PSYCDIS', 'PSYCDISX',
       'NACCAAAS', 'NACCAANX', 'NACCAC', 'NACCACEI',
       'NACCADEP', 'NACCAHTN', 'NACCAMD', 'NACCANGI', 'NACCAPSY',
       'NACCBETA', 'NACCCCBS', 'NACCDBMD', 'NACCDIUR', 'NACCEMD',
       'NACCEPMD', 'NACCHTNC', 'NACCLIPL', 'NACCNSD', 'NACCPDMD',
       'NACCVASD']
input_physical_feats = ['HEIGHT','WEIGHT', 'BPSYS', 'BPDIAS', 'HRATE',
        'VISION', 'VISCORR','VISWCORR', 'HEARING', 'HEARAID', 'HEARWAID',
        'NACCBMI']
input_exam_feats = ['FOCLSYM','FOCLSIGN','NACCNREX', 'NORMEXAM', 'DECSUB']
input_npi_feats = ['NPIQINF', 'NPIQINFX', 'DEL', 'DELSEV', 'HALL',
       'HALLSEV', 'AGIT', 'AGITSEV', 'DEPD', 'DEPDSEV', 'ANX', 'ANXSEV',
       'ELAT', 'ELATSEV', 'APA', 'APASEV', 'DISN', 'DISNSEV', 'IRR',
       'IRRSEV', 'MOT', 'MOTSEV', 'NITE', 'NITESEV', 'APP', 'APPSEV']
input_gds_feats = ['NOGDS', 'SATIS', 'DROPACT', 'EMPTY', 'BORED', 'SPIRITS', 'AFRAID',
       'HAPPY', 'HELPLESS', 'STAYHOME', 'MEMPROB', 'WONDRFUL', 'WRTHLESS',
       'ENERGY', 'HOPELESS', 'BETTER', 'NACCGDS']
input_faq_feats = ['BILLS', 'TAXES','SHOPPING', 'GAMES', 'STOVE',
        'MEALPREP', 'EVENTS', 'PAYATTN','REMDATES', 'TRAVEL']
input_scd_feats = ['DECSUB']
intput_np_feats = ['MMSEORDA',
       'MMSEORLO', 'PENTAGON', 'NACCMMSE', 'LOGIMEM', 'MEMUNITS',
       'MEMTIME', 'DIGIF', 'DIGIFLEN', 'DIGIB', 'DIGIBLEN', 'ANIMALS',
       'VEG', 'TRAILA', 'TRAILARR', 'TRAILALI', 'TRAILB', 'TRAILBRR',
       'TRAILBLI', 'BOSTON', 'MOCATOTS']
input_gene_feats = ['NACCAPOE']
label_feat = ['NACCUDSD']
# cdr_feats = ['MEMORY', 'ORIENT', 'JUDGMENT', 'COMMUN', 'HOMEHOBB', 'PERSCARE', 'CDRSUM', 'CDRGLOB']
UDS_selected_features = label_feat + input_gene_feats + intput_np_feats + input_scd_feats + input_faq_feats + input_gds_feats + input_npi_feats + input_exam_feats + input_physical_feats + input_patient_hist_feats + input_fam_hist_feats + input_copart_feats

# Helper functions - runs feature selection algs in one line

In [18]:
def helper_AE(train_df, test_df, e):
    model, scaler = AE(train_df, epochs = e)
    processed_AE_train = through_AE(train_df, model, scaler)
    processed_AE_test = through_AE(test_df, model, scaler)
    
    return processed_AE_train, processed_AE_test

In [19]:
def helper_SE(train_df, test_df, e):
    model, scaler = AE(train_df, epochs = e)
    processed_AE_train = through_AE(train_df, model, scaler)
    processed_AE_test = through_AE(test_df, model, scaler)
    
    return processed_AE_train, processed_AE_test

In [20]:
MRI_TRAIN, MRI_TRAIN_NAME, MRI_TEST, MRI_TEST_NAME = split_mri(53)
CURR_TRAIN, CURR_TEST = helper_AE(MRI_TRAIN, MRI_TEST, 10)



Train set shape: (1239, 156)
Test set shape: (414, 156)
(155,)
Epoch 1/10
78/78 [==============================] - 1s 2ms/step - loss: 0.9979 - accuracy: 0.0048
Epoch 2/10
78/78 [==============================] - 0s 2ms/step - loss: 0.9926 - accuracy: 0.0169
Epoch 3/10
78/78 [==============================] - 0s 2ms/step - loss: 0.9904 - accuracy: 0.0315
Epoch 4/10
78/78 [==============================] - 0s 2ms/step - loss: 0.9896 - accuracy: 0.0291
Epoch 5/10
78/78 [==============================] - 0s 2ms/step - loss: 0.9889 - accuracy: 0.0404
Epoch 6/10
78/78 [==============================] - 0s 2ms/step - loss: 0.9879 - accuracy: 0.0395
Epoch 7/10
78/78 [==============================] - 0s 2ms/step - loss: 0.9874 - accuracy: 0.0533
Epoch 8/10
78/78 [==============================] - 0s 2ms/step - loss: 0.9869 - accuracy: 0.0581
Epoch 9/10
78/78 [==============================] - 0s 2ms/step - loss: 0.9864 - accuracy: 0.0670
Epoch 10/10
78/78 [==============================] - 0s

# MRI

In [21]:
def run_mri_tests(seed):
    
    MRI_TRAIN, MRI_TRAIN_NAME, MRI_TEST, MRI_TEST_NAME = split_mri(seed)

    ## AUTOENCODER
    model,scaler = AE(MRI_TRAIN, epochs = 50)
    processed_AE_train = through_AE(MRI_TRAIN, model, scaler)
    processed_AE_test = through_AE(MRI_TEST, model, scaler)
    
    mri_ae_rf_acc, mri_ae_rf_tbl = random_forest(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])
    # return (mri_ae_rf_acc)
    mri_ae_lr_acc, mri_ae_lr_tbl = logistic_regression(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])
    mri_ae_nn_acc, mri_ae_nn_tbl = neural_network(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])
    
    ## PCA
    pca_train, pca_test = pca(train_df = MRI_TRAIN, test_df = MRI_TEST, var = 0.3)

    mri_pca_rf_acc, mri_pca_rf_tbl = random_forest(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])
    mri_pca_lr_acc, mri_pca_lr_tbl = logistic_regression( train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])
    mri_pca_nn_acc, mri_pca_nn_tbl = neural_network( train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])
    
    ## LASSO
    cols = lasso(df = MRI_TRAIN, al = 0.01, thresh = 0.05)

    mri_lasso_rf_acc, mri_lasso_rf_tbl = random_forest( train_df = MRI_TRAIN, test_df = MRI_TEST, cols = cols)
    mri_lasso_lr_acc, mri_lasso_lr_tbl = logistic_regression( train_df = MRI_TRAIN, test_df = MRI_TEST, cols = cols)
    mri_lasso_nn_acc, mri_lasso_nn_tbl = neural_network(train_df = MRI_TRAIN, test_df = MRI_TEST, cols = cols)
    
    ## SE
    mod, scaler = SDNN(MRI_TRAIN)

    processed_df_train = through_SDNN(MRI_TRAIN, mod, scaler)
    processed_df_test = through_SDNN(MRI_TEST, mod, scaler)

    processed_df_train.columns = processed_df_train.columns.astype(str)
    processed_df_test.columns = processed_df_test.columns.astype(str)

    mri_sdnn_rf_acc, mri_sdnn_rf_tbl = random_forest( train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])
    mri_sdnn_lr_acc, mri_sdnn_lr_tbl = logistic_regression(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])
    mri_sdnn_nn_acc, mri_sdnn_nn_tbl = neural_network( train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])
    
    
    
    import numpy as np
    pca_train, pca_test = pca(train_df = MRI_TRAIN, test_df = MRI_TEST)
    MULTICLASS = (
        mri_sdnn_rf_tbl,
        mri_sdnn_nn_tbl+1,
        mri_lasso_rf_tbl,
        mri_lasso_nn_tbl+1,
        mri_pca_rf_tbl,
        mri_pca_nn_tbl+1,
        mri_ae_rf_tbl,
        mri_ae_nn_tbl+1,
    )


    BINARY = (
        mri_sdnn_lr_tbl,
        mri_lasso_lr_tbl,
        mri_pca_lr_tbl,
        mri_ae_lr_tbl,
    )

    def do_thing(group, truth):

        # Combine input arrays into a single 2D array
        input_arrays = np.stack(group)

        def find_mode_with_tie_breaker(x):
            counts = np.bincount(x)
            max_count = np.max(counts)
            max_value = np.max([val for val, count in enumerate(counts) if count == max_count])
            return max_value

        # Apply the function along axis 0 (columns) of the input arrays
        voting_result = np.apply_along_axis(find_mode_with_tie_breaker, axis=0, arr=input_arrays)

        print(voting_result)

        # Calculate the number of matching elements
        matching_elements = np.sum(voting_result == truth)

        # Calculate the total number of elements
        total_elements = len(voting_result)

        # Calculate the percentage of matching values
        percentage_matching = (matching_elements / total_elements) * 100

        # print(f"Percentage of matching values: {percentage_matching:.2f}%")
        return percentage_matching

    multiclass_ensemble_acc = do_thing(MULTICLASS, MRI_TEST["NACCUDSD"])
    binary_ensemble_acc = do_thing(BINARY, (MRI_TEST["NACCUDSD"] != 1).astype(int))
    
    

    return [[mri_pca_lr_acc, mri_pca_rf_acc, mri_pca_nn_acc],
            [mri_ae_lr_acc, mri_ae_rf_acc, mri_ae_nn_acc],
           [mri_lasso_lr_acc, mri_lasso_rf_acc, mri_lasso_nn_acc],
            [mri_sdnn_lr_acc, mri_sdnn_rf_acc, mri_sdnn_nn_acc], multiclass_ensemble_acc, binary_ensemble_acc]
    
        
        
        

In [22]:
# run_mri_tests(42) achieves the following, which is similar-ish to current results.
"""
[[0.7995169082125604, 0.7028985507246377, 0.729468584060669],
 [0.7560386473429952, 0.6497584541062802, 0.6859903335571289],
 [0.785024154589372, 0.717391304347826, 0.7149758338928223],
 [0.7922705314009661, 0.714975845410628, 0.6980676054954529],
 74.8792270531401,
 80.43478260869566]
"""

'\n[[0.7995169082125604, 0.7028985507246377, 0.729468584060669],\n [0.7560386473429952, 0.6497584541062802, 0.6859903335571289],\n [0.785024154589372, 0.717391304347826, 0.7149758338928223],\n [0.7922705314009661, 0.714975845410628, 0.6980676054954529],\n 74.8792270531401,\n 80.43478260869566]\n'

# UDS

In [23]:
def run_uds_tests(seed):
    
    UDS_TRAIN, UDS_TRAIN_NAME, UDS_TEST, UDS_TEST_NAME = split_uds(seed)

    pca_train, pca_test = pca(train_df = UDS_TRAIN, test_df = UDS_TEST)

    uds_pca_rf_acc, uds_pca_rf_tbl = random_forest(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])
    uds_pca_lr_acc, uds_pca_lr_tbl = logistic_regression(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])
    uds_pca_nn_acc, uds_pca_nn_tbl = neural_network(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])

    cols = lasso(df = UDS_TRAIN, al = 0.01, thresh = 0.04)
    uds_lasso_rf_acc, uds_lasso_rf_tbl = random_forest(train_df = UDS_TRAIN, test_df = UDS_TEST, cols = cols)
    uds_lasso_lr_acc, uds_lasso_lr_tbl = logistic_regression(train_df = UDS_TRAIN, test_df = UDS_TEST, cols = cols)
    uds_lasso_nn_acc, uds_lasso_nn_tbl = neural_network(train_df = UDS_TRAIN, test_df = UDS_TEST, cols = cols)

    mod,scaler = SDNN(UDS_TRAIN)
    processed_df_train = through_SDNN(UDS_TRAIN, mod, scaler)
    processed_df_test = through_SDNN(UDS_TEST, mod, scaler)
    uds_sdnn_lr_acc, uds_sdnn_lr_tbl = logistic_regression(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])
    uds_sdnn_rf_acc, uds_sdnn_rf_tbl = random_forest(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])
    uds_sdnn_nn_acc, uds_sdnn_nn_tbl = neural_network(train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1]) 

    model, scaler = AE(UDS_TRAIN, epochs = 50)
    processed_AE_train = through_AE(UDS_TRAIN, model, scaler)
    processed_AE_test = through_AE(UDS_TEST, model, scaler)
    uds_ae_lr_acc, uds_ae_lr_tbl = logistic_regression(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])
    uds_ae_nn_acc, uds_ae_nn_tbl = neural_network(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])
    uds_ae_rf_acc, uds_ae_rf_tbl = random_forest(train_df = processed_AE_train, test_df = processed_AE_test, cols = processed_AE_train.columns[:-1])
    
    import numpy as np
    pca_train, pca_test = pca(train_df = UDS_TRAIN, test_df = UDS_TEST)
    MULTICLASS = (
        uds_sdnn_rf_tbl,
        uds_sdnn_nn_tbl+1,
        uds_lasso_rf_tbl,
        uds_lasso_nn_tbl+1,
        uds_pca_rf_tbl,
        uds_pca_nn_tbl+1,
        uds_ae_rf_tbl,
        uds_ae_nn_tbl+1,
    )


    BINARY = (
        uds_sdnn_lr_tbl,
        uds_lasso_lr_tbl,
        uds_pca_lr_tbl,
        uds_ae_lr_tbl,
    )

    def do_thing(group, truth):

        # Combine input arrays into a single 2D array
        input_arrays = np.stack(group)

        def find_mode_with_tie_breaker(x):
            counts = np.bincount(x)
            max_count = np.max(counts)
            max_value = np.max([val for val, count in enumerate(counts) if count == max_count])
            return max_value

        # Apply the function along axis 0 (columns) of the input arrays
        voting_result = np.apply_along_axis(find_mode_with_tie_breaker, axis=0, arr=input_arrays)

        print(voting_result)

        # Calculate the number of matching elements
        matching_elements = np.sum(voting_result == truth)

        # Calculate the total number of elements
        total_elements = len(voting_result)

        # Calculate the percentage of matching values
        percentage_matching = (matching_elements / total_elements) * 100

        return percentage_matching

    multiclass_ensemble_acc = do_thing(MULTICLASS, pca_test["NACCUDSD"])
    binary_ensemble_acc = do_thing(BINARY, (pca_test["NACCUDSD"] != 1).astype(int))
    
    
    return [
           [uds_pca_lr_acc, uds_pca_rf_acc, uds_pca_nn_acc],
            [uds_ae_lr_acc, uds_ae_rf_acc, uds_ae_nn_acc],
           [uds_lasso_lr_acc, uds_lasso_rf_acc, uds_lasso_nn_acc],
            [uds_sdnn_lr_acc, uds_sdnn_rf_acc, uds_sdnn_nn_acc], multiclass_ensemble_acc, binary_ensemble_acc]
    
        
        
        

In [24]:
# run_uds_tests(42) gets
"""
[[0.8574081716879901, 0.7930251754023937, 0.7983904480934143],
 [0.7263722657862154, 0.6591002888980603, 0.7026413679122925],
 [0.8609162195625258, 0.7998349153941395, 0.8025175333023071],
 [0.8801073049938093, 0.819026000825423, 0.8130416870117188],
 81.61370202228643,
 87.53611225753198]
"""

'\n[[0.8574081716879901, 0.7930251754023937, 0.7983904480934143],\n [0.7263722657862154, 0.6591002888980603, 0.7026413679122925],\n [0.8609162195625258, 0.7998349153941395, 0.8025175333023071],\n [0.8801073049938093, 0.819026000825423, 0.8130416870117188],\n 81.61370202228643,\n 87.53611225753198]\n'

# CSF

In [25]:
def run_csf_tests(seed):
    
    CSF_TRAIN, CSF_TRAIN_NAME, CSF_TEST, CSF_TEST_NAME = split_csf(seed)
    
    csf_lr_acc, csf_lr_tbl = logistic_regression(train_df = CSF_TRAIN, test_df = CSF_TEST, cols = ['CSFABETA', 'CSFPTAU', 'CSFTTAU'])
    csf_rf_acc, csf_rf_tbl = random_forest( train_df = CSF_TRAIN, test_df = CSF_TEST, cols = ['CSFABETA', 'CSFPTAU', 'CSFTTAU'])
    csf_nn_acc, csf_nn_tbl =  neural_network( train_df = CSF_TRAIN, test_df = CSF_TEST, cols = ['CSFABETA', 'CSFPTAU', 'CSFTTAU'])
    
    import numpy as np

    MULTICLASS = (
        csf_nn_tbl+1,
        csf_rf_tbl
    )


    def do_thing(group, truth):

        # Combine input arrays into a single 2D array
        input_arrays = np.stack(group)

        def find_mode_with_tie_breaker(x):
            counts = np.bincount(x)
            max_count = np.max(counts)
            max_value = np.max([val for val, count in enumerate(counts) if count == max_count])
            return max_value

        # Apply the function along axis 0 (columns) of the input arrays
        voting_result = np.apply_along_axis(find_mode_with_tie_breaker, axis=0, arr=input_arrays)

        print(voting_result)

        # Calculate the number of matching elements
        matching_elements = np.sum(voting_result == truth)

        # Calculate the total number of elements
        total_elements = len(voting_result)

        # Calculate the percentage of matching values
        percentage_matching = (matching_elements / total_elements) * 100

        return percentage_matching

    multiclass_ensemble_acc = do_thing(MULTICLASS, CSF_TEST['NACCUDSD'])

    
    return [[csf_lr_acc, csf_rf_acc, csf_nn_acc], multiclass_ensemble_acc]


        
        
        

In [26]:
# run_csf_tests(42) gets
"""
[[0.8169761273209549, 0.7374005305039788, 0.7824933528900146],
 76.39257294429707]
"""

'\n[[0.8169761273209549, 0.7374005305039788, 0.7824933528900146],\n 76.39257294429707]\n'

# Simple Fusion

In [27]:
def run_simple_fusion_tests(seed):
    
    UDS_MRI_TRAIN, UDS_MRI_TRAIN_NAME, UDS_MRI_TEST, UDS_MRI_TEST_NAME = split_uds_mri(seed)
    
    uds_mri_s_lr_acc, uds_mri_s_lr_tbl = logistic_regression(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_MRI_TRAIN.columns[:-1])
    uds_mri_s_rf_acc, uds_mri_s_rf_tbl = random_forest(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_MRI_TRAIN.columns[:-1])
    uds_mri_s_nn_acc, uds_mri_s_nn_tbl = neural_network(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_MRI_TRAIN.columns[:-1])
    

    import numpy as np

    MULTICLASS = (
        uds_mri_s_nn_tbl+1,
        uds_mri_s_rf_tbl
    )


    def do_thing(group, truth):

        # Combine input arrays into a single 2D array
        input_arrays = np.stack(group)

        def find_mode_with_tie_breaker(x):
            counts = np.bincount(x)
            max_count = np.max(counts)
            max_value = np.max([val for val, count in enumerate(counts) if count == max_count])
            return max_value

        # Apply the function along axis 0 (columns) of the input arrays
        voting_result = np.apply_along_axis(find_mode_with_tie_breaker, axis=0, arr=input_arrays)

        print(voting_result)

        # Calculate the number of matching elements
        matching_elements = np.sum(voting_result == truth)

        # Calculate the total number of elements
        total_elements = len(voting_result)

        # Calculate the percentage of matching values
        percentage_matching = (matching_elements / total_elements) * 100

        return percentage_matching

    multiclass_ensemble_acc = do_thing(MULTICLASS, UDS_MRI_TEST['NACCUDSD'])
    
    return [[uds_mri_s_lr_acc, uds_mri_s_rf_acc, uds_mri_s_nn_acc], multiclass_ensemble_acc]
        
    
        
        
        

In [28]:
UDS_MRI_TRAIN, UDS_MRI_TRAIN_NAME, UDS_MRI_TEST, UDS_MRI_TEST_NAME = split_uds_mri(seed)
uds_mri_s_rf_tbl = random_forest(give_table = True, train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_MRI_TRAIN.columns[:-1])

NameError: name 'seed' is not defined

In [29]:
# run_simple_fusion_tests(42) gets 
"""
[[0.7927170868347339, 0.8263305322128851, 0.8067227005958557],
 83.19327731092437]
 """

'\n[[0.7927170868347339, 0.8263305322128851, 0.8067227005958557],\n 83.19327731092437]\n '

In [30]:
def run_early_fusion_tests(seed):
    
    UDS_MRI_TRAIN, UDS_MRI_TRAIN_NAME, UDS_MRI_TEST, UDS_MRI_TEST_NAME = split_uds_mri(seed)
    
    pca_train, pca_test = pca(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST)
    uds_mri_e_pca_rf_acc, uds_mri_e_pca_rf_tbl = random_forest( train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])
    uds_mri_e_pca_lr_acc, uds_mri_e_pca_lr_tbl = logistic_regression( train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8])
    uds_mri_e_pca_nn_acc, uds_mri_e_pca_nn_tbl = neural_network(train_df = pca_train, test_df = pca_test, cols = pca_train.columns[:8], epochs = 15)

    them_cols = lasso(df = UDS_MRI_TRAIN, al = 0.05, thresh = 0.05) # al & thresh .05 worked best
    uds_mri_e_lasso_lr_acc, uds_mri_e_lasso_lr_tbl = logistic_regression( train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = them_cols)
    uds_mri_e_lasso_rf_acc, uds_mri_e_lasso_rf_tbl = random_forest( train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = lasso(df = UDS_MRI_TRAIN, al = 2 / 100, thresh = 1 / 100))
    uds_mri_e_lasso_nn_acc, uds_mri_e_lasso_nn_tbl = neural_network( train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = them_cols)

    mod, scaler = SDNN(UDS_MRI_TRAIN)
    processed_df_train = through_SDNN(UDS_MRI_TRAIN, mod, scaler)
    processed_df_test = through_SDNN(UDS_MRI_TEST, mod, scaler)
    processed_df_train.columns = processed_df_train.columns.astype(str)
    processed_df_test.columns = processed_df_test.columns.astype(str)
    uds_mri_e_sdnn_rf_acc, uds_mri_e_sdnn_rf_tbl = random_forest( train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])
    uds_mri_e_sdnn_lr_acc, uds_mri_e_sdnn_lr_tbl = logistic_regression( train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])
    uds_mri_e_sdnn_nn_acc, uds_mri_e_sdnn_nn_tbl = neural_network( train_df = processed_df_train, test_df = processed_df_test, epochs = 3, cols = processed_df_train.columns[:-1])

    mod, scaler = AE(UDS_MRI_TRAIN, epochs = 50)
    processed_df_train = through_AE(UDS_MRI_TRAIN, mod, scaler)
    processed_df_test = through_AE(UDS_MRI_TEST, mod, scaler)
    processed_df_train.columns = processed_df_train.columns.astype(str)
    processed_df_test.columns = processed_df_test.columns.astype(str)
    uds_mri_e_ae_lr_acc, uds_mri_e_ae_lr_tbl = logistic_regression( train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])
    uds_mri_e_ae_rf_acc, uds_mri_e_ae_rf_tbl = random_forest( train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])
    uds_mri_e_ae_nn_acc, uds_mri_e_ae_nn_tbl = neural_network( train_df = processed_df_train, test_df = processed_df_test, cols = processed_df_train.columns[:-1])

    import numpy as np
    pca_train, pca_test = pca(train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST)
    MULTICLASS = (
        uds_mri_e_ae_nn_tbl+1,
        uds_mri_e_ae_rf_tbl,
        uds_mri_e_lasso_nn_tbl+1,
        uds_mri_e_lasso_rf_tbl,
        uds_mri_e_pca_nn_tbl+1,
        uds_mri_e_pca_rf_tbl,
        uds_mri_e_sdnn_nn_tbl+1,
        uds_mri_e_sdnn_rf_tbl,
    )

    BINARY = (
        uds_mri_e_ae_lr_tbl,
        uds_mri_e_lasso_lr_tbl,
        uds_mri_e_pca_lr_tbl,
        uds_mri_e_sdnn_lr_tbl
    )

    def do_thing(group, truth):

        # Combine input arrays into a single 2D array
        input_arrays = np.stack(group)

        def find_mode_with_tie_breaker(x):
            counts = np.bincount(x)
            max_count = np.max(counts)
            max_value = np.max([val for val, count in enumerate(counts) if count == max_count])
            return max_value

        # Apply the function along axis 0 (columns) of the input arrays
        voting_result = np.apply_along_axis(find_mode_with_tie_breaker, axis=0, arr=input_arrays)

        print(voting_result)

        # Calculate the number of matching elements
        matching_elements = np.sum(voting_result == truth)

        # Calculate the total number of elements
        total_elements = len(voting_result)

        # Calculate the percentage of matching values
        percentage_matching = (matching_elements / total_elements) * 100

        return percentage_matching

    multiclass_ensemble_acc = do_thing(MULTICLASS, pca_test["NACCUDSD"])
    binary_ensemble_acc = do_thing(BINARY, (pca_test["NACCUDSD"] != 1).astype(int))
    
    return [[uds_mri_e_pca_lr_acc, uds_mri_e_pca_rf_acc, uds_mri_e_pca_nn_acc],
            [uds_mri_e_ae_lr_acc, uds_mri_e_ae_rf_acc, uds_mri_e_ae_nn_acc],
            [uds_mri_e_lasso_lr_acc, uds_mri_e_lasso_rf_acc, uds_mri_e_lasso_nn_acc],
           [uds_mri_e_sdnn_lr_acc, uds_mri_e_sdnn_rf_acc, uds_mri_e_sdnn_nn_acc],
             multiclass_ensemble_acc, binary_ensemble_acc]
        
    
        
        


In [31]:
# run_early_fusion_tests(42) gets
"""
[[0.8375350140056023, 0.7815126050420168, 0.7703081369400024],
 [0.7983193277310925, 0.7310924369747899, 0.756302535533905],
 [0.8879551820728291, 0.8319327731092437, 0.8347339034080505],
 [0.8823529411764706, 0.7927170868347339, 0.7983193397521973],
 80.3921568627451,
 88.51540616246498]
"""

'\n[[0.8375350140056023, 0.7815126050420168, 0.7703081369400024],\n [0.7983193277310925, 0.7310924369747899, 0.756302535533905],\n [0.8879551820728291, 0.8319327731092437, 0.8347339034080505],\n [0.8823529411764706, 0.7927170868347339, 0.7983193397521973],\n 80.3921568627451,\n 88.51540616246498]\n'

In [42]:
def run_late_fusion_tests(seed):
    # This is a bit weird, but due to my data pipeline the merged UDS_MRI doesn't have the same features as UDS and MRI.
    # That was to preserve more data in each dataset. So here we are just finding columns in common.
    
    UDS_MRI_TRAIN, UDS_MRI_TRAIN_NAME, UDS_MRI_TEST, UDS_MRI_TEST_NAME = split_uds_mri(seed)
    UDS_TRAIN, UDS_TRAIN_NAME, UDS_TEST, UDS_TEST_NAME = split_uds(seed)
    MRI_TRAIN, MRI_TRAIN_NAME, MRI_TEST, MRI_TEST_NAME = split_mri(seed)

    UDS_cols_in_merged = [x for x in UDS_MRI_TRAIN.columns if x not in input_mri_feats]
    UDS_cols_use = [x for x in UDS_cols_in_merged if x in UDS_TRAIN.columns]

    MRI_cols_in_merged = [x for x in UDS_MRI_TRAIN.columns if x not in UDS_cols_in_merged]
    MRI_cols_use = [x for x in MRI_cols_in_merged if x in MRI_TRAIN.columns] + ['NACCUDSD']
    
    UDS_mod, UDS_scaler = SDNN(UDS_TRAIN[UDS_cols_use])
    MRI_mod, MRI_scaler = SDNN(MRI_TRAIN[MRI_cols_use])
    processed_df_UDS_train = through_SDNN(UDS_MRI_TRAIN[UDS_cols_use], UDS_mod, UDS_scaler)
    processed_df_UDS_test = through_SDNN(UDS_MRI_TEST[UDS_cols_use], UDS_mod, UDS_scaler)
    processed_df_MRI_train = through_SDNN(UDS_MRI_TRAIN[MRI_cols_use], MRI_mod, MRI_scaler)
    processed_df_MRI_test = through_SDNN(UDS_MRI_TEST[MRI_cols_use], MRI_mod , MRI_scaler)
    merged_sdnn_train = processed_df_UDS_train[processed_df_UDS_train.columns[:-1]].merge(processed_df_MRI_train[processed_df_MRI_train.columns[:-1]], how = "outer", left_index=True, right_index=True)
    merged_sdnn_train['NACCUDSD'] = processed_df_UDS_train['NACCUDSD']
    merged_sdnn_train
    merged_sdnn_test = processed_df_UDS_test[processed_df_UDS_test.columns[:-1]].merge(processed_df_MRI_test[processed_df_MRI_test.columns[:-1]], how = "outer", left_index=True, right_index=True)
    merged_sdnn_test['NACCUDSD'] = processed_df_UDS_test['NACCUDSD']
    uds_mri_l_sdnn_lr_acc, uds_mri_l_sdnn_lr_tbl = logistic_regression( train_df = merged_sdnn_train, test_df = merged_sdnn_test, cols = merged_sdnn_train.columns[:-1])
    uds_mri_l_sdnn_rf_acc, uds_mri_l_sdnn_rf_tbl = random_forest( train_df = merged_sdnn_train, test_df = merged_sdnn_test, cols = merged_sdnn_train.columns[:-1])
    uds_mri_l_sdnn_nn_acc, uds_mri_l_sdnn_nn_tbl = neural_network( train_df = merged_sdnn_train, test_df = merged_sdnn_test, cols = merged_sdnn_train.columns[:-1])

    UDS_mod, UDS_scaler = AE(UDS_TRAIN[UDS_cols_use], epochs = 50)
    MRI_mod, MRI_scaler = AE(MRI_TRAIN[MRI_cols_use], epochs = 50)
    processed_df_UDS_train = through_AE(UDS_MRI_TRAIN[UDS_cols_use], UDS_mod, UDS_scaler)
    processed_df_UDS_test = through_AE(UDS_MRI_TEST[UDS_cols_use], UDS_mod, UDS_scaler)
    processed_df_MRI_train = through_AE(UDS_MRI_TRAIN[MRI_cols_use], MRI_mod, MRI_scaler)
    processed_df_MRI_test = through_AE(UDS_MRI_TEST[MRI_cols_use], MRI_mod, MRI_scaler)
    merged_ae_train = processed_df_UDS_train[processed_df_UDS_train.columns[:-1]].merge(processed_df_MRI_train[processed_df_MRI_train.columns[:-1]], how = "outer", left_index=True, right_index=True)
    merged_ae_train['NACCUDSD'] = processed_df_UDS_train['NACCUDSD']
    merged_ae_test = processed_df_UDS_test[processed_df_UDS_test.columns[:-1]].merge(processed_df_MRI_test[processed_df_MRI_test.columns[:-1]], how = "outer", left_index=True, right_index=True)
    merged_ae_test['NACCUDSD'] = processed_df_UDS_test['NACCUDSD']

    uds_mri_l_ae_lr_acc, uds_mri_l_ae_lr_tbl = logistic_regression( train_df = merged_ae_train, test_df = merged_ae_test, cols = merged_ae_train.columns[:-1])
    uds_mri_l_ae_rf_acc, uds_mri_l_ae_rf_tbl = random_forest( train_df = merged_ae_train, test_df = merged_ae_test, cols = merged_ae_train.columns[:-1])
    uds_mri_l_ae_nn_acc, uds_mri_l_ae_nn_tbl = neural_network( train_df = merged_ae_train, test_df = merged_ae_test, cols = merged_ae_train.columns[:-1], epochs = 50)
    
    pca_UDS_train, pca_UDS_test = pca(train_df = UDS_MRI_TRAIN[UDS_cols_use], test_df = UDS_MRI_TEST[UDS_cols_use])
    pca_MRI_train, pca_MRI_test = pca(train_df = UDS_MRI_TRAIN[MRI_cols_use], test_df = UDS_MRI_TEST[MRI_cols_use])
    merged_pca_train = pca_MRI_train[pca_MRI_train.columns[:8]].merge(pca_UDS_train[pca_UDS_train.columns[:8]], how = "outer", left_index=True, right_index=True)
    merged_pca_train['NACCUDSD'] = pca_MRI_train['NACCUDSD']
    merged_pca_train
    merged_pca_test = pca_MRI_test[pca_MRI_test.columns[:8]].merge(pca_UDS_test[pca_UDS_test.columns[:8]], how = "outer", left_index=True, right_index=True)
    merged_pca_test['NACCUDSD'] = pca_MRI_test['NACCUDSD']
    merged_pca_test
    uds_mri_l_pca_rf_acc, uds_mri_l_pca_rf_tbl = random_forest( train_df = merged_pca_train, test_df = merged_pca_test, cols = merged_pca_train.columns[:-1])
    uds_mri_l_pca_lr_acc, uds_mri_l_pca_lr_tbl = logistic_regression( train_df = merged_pca_train, test_df = merged_pca_test, cols = merged_pca_train.columns[:-1])
    uds_mri_l_pca_nn_acc, uds_mri_l_pca_nn_tbl = neural_network( train_df = merged_pca_train, test_df = merged_pca_test, cols = merged_pca_train.columns[:-1], epochs = 20)

    
    UDS_cols = lasso(df = UDS_TRAIN, al = 0.01, thresh = 0.04)
    UDS_cols
    MRI_cols = lasso(df = MRI_TRAIN, al = 0.01, thresh = 0.04)
    MRI_cols
    uds_mri_l_lasso_lr_acc, uds_mri_l_lasso_lr_tbl = logistic_regression( train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_cols+MRI_cols)
    uds_mri_l_lasso_rf_acc, uds_mri_l_lasso_rf_tbl = random_forest( train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_cols+MRI_cols)
    uds_mri_l_lasso_nn_acc, uds_mri_l_lasso_nn_tbl = neural_network( train_df = UDS_MRI_TRAIN, test_df = UDS_MRI_TEST, cols = UDS_cols+MRI_cols, epochs = 15)

    
    

    print(uds_mri_l_sdnn_lr_acc)
    
    import numpy as np

    MULTICLASS = (
        uds_mri_l_ae_nn_tbl+1,
        uds_mri_l_ae_rf_tbl,
        uds_mri_l_lasso_nn_tbl+1,
        uds_mri_l_lasso_rf_tbl,
        uds_mri_l_pca_nn_tbl+1,
        uds_mri_l_pca_rf_tbl,
        uds_mri_l_sdnn_nn_tbl+1,
        uds_mri_l_sdnn_rf_tbl,
    )

    BINARY = (
        uds_mri_l_ae_lr_tbl,
        uds_mri_l_lasso_lr_tbl,
        uds_mri_l_pca_lr_tbl,
        uds_mri_l_sdnn_lr_tbl
    )

    def do_thing(group, truth):

        # Combine input arrays into a single 2D array
        input_arrays = np.stack(group)

        def find_mode_with_tie_breaker(x):
            counts = np.bincount(x)
            max_count = np.max(counts)
            max_value = np.max([val for val, count in enumerate(counts) if count == max_count])
            return max_value

        # Apply the function along axis 0 (columns) of the input arrays
        voting_result = np.apply_along_axis(find_mode_with_tie_breaker, axis=0, arr=input_arrays)

        print(voting_result)

        # Calculate the number of matching elements
        matching_elements = np.sum(voting_result == truth)

        # Calculate the total number of elements
        total_elements = len(voting_result)

        # Calculate the percentage of matching values
        percentage_matching = (matching_elements / total_elements) * 100

        return percentage_matching

    multiclass_ensemble_acc = do_thing(MULTICLASS, merged_pca_test['NACCUDSD'])
    binary_ensemble_acc = do_thing(BINARY, (merged_pca_test['NACCUDSD'] != 1).astype(int))
    
    return [[uds_mri_l_pca_lr_acc, uds_mri_l_pca_rf_acc, uds_mri_l_pca_nn_acc],
[uds_mri_l_ae_lr_acc, uds_mri_l_ae_rf_acc, uds_mri_l_ae_nn_acc],
[uds_mri_l_lasso_lr_acc, uds_mri_l_lasso_rf_acc, uds_mri_l_lasso_nn_acc],
[uds_mri_l_sdnn_lr_acc, uds_mri_l_sdnn_rf_acc, uds_mri_l_sdnn_nn_acc], multiclass_ensemble_acc, binary_ensemble_acc]

In [46]:
seed = 4829
UDS_MRI_TRAIN, UDS_MRI_TRAIN_NAME, UDS_MRI_TEST, UDS_MRI_TEST_NAME = split_uds_mri(seed)
UDS_TRAIN, UDS_TRAIN_NAME, UDS_TEST, UDS_TEST_NAME = split_uds(seed)
MRI_TRAIN, MRI_TRAIN_NAME, MRI_TEST, MRI_TEST_NAME = split_mri(seed)

UDS_cols_in_merged = [x for x in UDS_MRI_TRAIN.columns if x not in input_mri_feats]
UDS_cols_use = [x for x in UDS_cols_in_merged if x in UDS_TRAIN.columns]

MRI_cols_in_merged = [x for x in UDS_MRI_TRAIN.columns if x not in UDS_cols_in_merged]
MRI_cols_use = [x for x in MRI_cols_in_merged if x in MRI_TRAIN.columns] + ['NACCUDSD']


UDS_mod, UDS_scaler = SDNN(UDS_TRAIN[UDS_cols_use])
MRI_mod, MRI_scaler = SDNN(MRI_TRAIN[MRI_cols_use])
processed_df_UDS_train = through_SDNN(UDS_MRI_TRAIN[UDS_cols_use], UDS_mod, UDS_scaler)
processed_df_UDS_test = through_SDNN(UDS_MRI_TEST[UDS_cols_use], UDS_mod, UDS_scaler)
processed_df_MRI_train = through_SDNN(UDS_MRI_TRAIN[MRI_cols_use], MRI_mod, MRI_scaler)
processed_df_MRI_test = through_SDNN(UDS_MRI_TEST[MRI_cols_use], MRI_mod , MRI_scaler)
merged_sdnn_train = processed_df_UDS_train[processed_df_UDS_train.columns[:-1]].merge(processed_df_MRI_train[processed_df_MRI_train.columns[:-1]], how = "outer", left_index=True, right_index=True)
merged_sdnn_train['NACCUDSD'] = processed_df_UDS_train['NACCUDSD']
merged_sdnn_train
merged_sdnn_test = processed_df_UDS_test[processed_df_UDS_test.columns[:-1]].merge(processed_df_MRI_test[processed_df_MRI_test.columns[:-1]], how = "outer", left_index=True, right_index=True)
merged_sdnn_test['NACCUDSD'] = processed_df_UDS_test['NACCUDSD']
uds_mri_l_sdnn_lr_acc, uds_mri_l_sdnn_lr_tbl = logistic_regression( train_df = merged_sdnn_train, test_df = merged_sdnn_test, cols = merged_sdnn_train.columns[:-1])
uds_mri_l_sdnn_rf_acc, uds_mri_l_sdnn_rf_tbl = random_forest( train_df = merged_sdnn_train, test_df = merged_sdnn_test, cols = merged_sdnn_train.columns[:-1])
uds_mri_l_sdnn_nn_acc, uds_mri_l_sdnn_nn_tbl = neural_network( train_df = merged_sdnn_train, test_df = merged_sdnn_test, cols = merged_sdnn_train.columns[:-1])

uds_mri_l_sdnn_lr_acc, uds_mri_l_sdnn_rf_acc, uds_mri_l_sdnn_nn_acc

Train set shape: (1068, 265)
Test set shape: (357, 265)
Train set shape: (14535, 110)
Test set shape: (4846, 110)
Train set shape: (1239, 156)
Test set shape: (414, 156)
Epoch 1/10
909/909 [==============================] - 2s 2ms/step - loss: 0.5659 - accuracy: 0.7928
Epoch 2/10
909/909 [==============================] - 1s 2ms/step - loss: 0.4647 - accuracy: 0.8261
Epoch 3/10
909/909 [==============================] - 2s 2ms/step - loss: 0.4360 - accuracy: 0.8380
Epoch 4/10
909/909 [==============================] - 1s 2ms/step - loss: 0.4144 - accuracy: 0.8446
Epoch 5/10
909/909 [==============================] - 2s 2ms/step - loss: 0.3982 - accuracy: 0.8535
Epoch 6/10
909/909 [==============================] - 2s 2ms/step - loss: 0.3799 - accuracy: 0.8567
Epoch 7/10
909/909 [==============================] - 2s 2ms/step - loss: 0.3639 - accuracy: 0.8632
Epoch 8/10
909/909 [==============================] - 2s 2ms/step - loss: 0.3488 - accuracy: 0.8718
Epoch 9/10
909/909 [==========

(0.9243697478991597, 0.8823529411764706, 0.8935574293136597)

In [43]:
run_late_fusion_tests(42)

Train set shape: (1068, 265)
Test set shape: (357, 265)
Train set shape: (14535, 110)
Test set shape: (4846, 110)
Train set shape: (1239, 156)
Test set shape: (414, 156)
Epoch 1/10
909/909 [==============================] - 2s 2ms/step - loss: 0.5955 - accuracy: 0.7777
Epoch 2/10
909/909 [==============================] - 2s 2ms/step - loss: 0.4791 - accuracy: 0.8196
Epoch 3/10
909/909 [==============================] - 1s 2ms/step - loss: 0.4435 - accuracy: 0.8330
Epoch 4/10
909/909 [==============================] - 2s 2ms/step - loss: 0.4216 - accuracy: 0.8422
Epoch 5/10
909/909 [==============================] - 2s 2ms/step - loss: 0.3988 - accuracy: 0.8476
Epoch 6/10
909/909 [==============================] - 2s 2ms/step - loss: 0.3848 - accuracy: 0.8531
Epoch 7/10
909/909 [==============================] - 2s 2ms/step - loss: 0.3709 - accuracy: 0.8575
Epoch 8/10
909/909 [==============================] - 2s 2ms/step - loss: 0.3548 - accuracy: 0.8630
Epoch 9/10
909/909 [==========

C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9047619047619048
0.8543417366946778
Epoch 1/10
34/34 [==============================] - 1s 2ms/step - loss: 1.0488 - accuracy: 0.6498
Epoch 2/10
34/34 [==============================] - 0s 2ms/step - loss: 0.4918 - accuracy: 0.8287
Epoch 3/10
34/34 [==============================] - 0s 2ms/step - loss: 0.3835 - accuracy: 0.8652
Epoch 4/10
34/34 [==============================] - 0s 2ms/step - loss: 0.3450 - accuracy: 0.8830
Epoch 5/10
34/34 [==============================] - 0s 2ms/step - loss: 0.3296 - accuracy: 0.8867
Epoch 6/10
34/34 [==============================] - 0s 2ms/step - loss: 0.3189 - accuracy: 0.8904
Epoch 7/10
34/34 [==============================] - 0s 1ms/step - loss: 0.3110 - accuracy: 0.8942
Epoch 8/10
34/34 [==============================] - 0s 2ms/step - loss: 0.3052 - accuracy: 0.8886
Epoch 9/10
34/34 [==============================] - 0s 1ms/step - loss: 0.3012 - accuracy: 0.8961
Epoch 10/10
34/34 [==============================] - 0s 2ms/step - loss: 0.2939 

C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.295e+03, tolerance: 2.788e+00
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.133e+02, tolerance: 2.098e-01
  model = cd_fast.enet_coordinate_descent(


0.8683473389355743
0.8095238095238095
Epoch 1/15
34/34 [==============================] - 1s 2ms/step - loss: 0.9091 - accuracy: 0.6564
Epoch 2/15
34/34 [==============================] - 0s 2ms/step - loss: 0.6312 - accuracy: 0.7800
Epoch 3/15
34/34 [==============================] - 0s 2ms/step - loss: 0.5219 - accuracy: 0.8062
Epoch 4/15
34/34 [==============================] - 0s 2ms/step - loss: 0.4804 - accuracy: 0.8137
Epoch 5/15
34/34 [==============================] - 0s 2ms/step - loss: 0.4544 - accuracy: 0.8221
Epoch 6/15
34/34 [==============================] - 0s 2ms/step - loss: 0.4346 - accuracy: 0.8436
Epoch 7/15
34/34 [==============================] - 0s 2ms/step - loss: 0.4181 - accuracy: 0.8418
Epoch 8/15
34/34 [==============================] - 0s 2ms/step - loss: 0.4010 - accuracy: 0.8567
Epoch 9/15
34/34 [==============================] - 0s 2ms/step - loss: 0.3843 - accuracy: 0.8661
Epoch 10/15
34/34 [==============================] - 0s 2ms/step - loss: 0.3715 

[[0.8571428571428571, 0.7983193277310925, 0.8151260614395142],
 [0.6246498599439776, 0.6974789915966386, 0.686274528503418],
 [0.8683473389355743, 0.8095238095238095, 0.8179271817207336],
 [0.9047619047619048, 0.8543417366946778, 0.8599439859390259],
 83.4733893557423,
 88.79551820728291]

# please work my sanity is running out

In [34]:
ALL_DATA = {}

for i in range(50):
    YUH = np.random.randint(100000)
    
    some_inner_scoped_variable = {"UDS": [],
           "MRI": [],
           "CSF":[],
           "Simple Fusion": [],
           "Early Fusion": [],
           "Late Fusion": []}
    some_inner_scoped_variable["UDS"] = run_uds_tests(YUH)
    some_inner_scoped_variable["MRI"] = run_mri_tests(YUH)
    some_inner_scoped_variable["CSF"] = run_csf_tests(YUH)
    some_inner_scoped_variable["Simple Fusion"] = run_simple_fusion_tests(YUH)
    some_inner_scoped_variable["Late Fusion"] = run_late_fusion_tests(YUH)
    some_inner_scoped_variable["Early Fusion"] = run_early_fusion_tests(YUH)
    
    ALL_DATA[YUH] = some_inner_scoped_variable
    
    import json
    with open("my_dict.json", "w") as f:
        json.dump(ALL_DATA, f, indent=4)
    
    

Train set shape: (14535, 110)
Test set shape: (4846, 110)
0.7781675608749484
0.8345026826248453
Epoch 1/10
455/455 [==============================] - 1s 2ms/step - loss: 0.6385 - accuracy: 0.7690
Epoch 2/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5774 - accuracy: 0.7852
Epoch 3/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5703 - accuracy: 0.7858
Epoch 4/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5653 - accuracy: 0.7862
Epoch 5/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5629 - accuracy: 0.7885
Epoch 6/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5590 - accuracy: 0.7856
Epoch 7/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5563 - accuracy: 0.7868
Epoch 8/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5547 - accuracy: 0.7884
Epoch 9/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5531 - accuracy: 0.7889
Epoc

C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.715e+02, tolerance: 2.787e+00
  model = cd_fast.enet_coordinate_descent(


0.8035493190260008
0.8594717292612464
Epoch 1/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5841 - accuracy: 0.7960
Epoch 2/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5190 - accuracy: 0.8125
Epoch 3/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5106 - accuracy: 0.8127
Epoch 4/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5047 - accuracy: 0.8111
Epoch 5/10
455/455 [==============================] - 1s 2ms/step - loss: 0.5001 - accuracy: 0.8140
Epoch 6/10
455/455 [==============================] - 1s 2ms/step - loss: 0.4987 - accuracy: 0.8129
Epoch 7/10
455/455 [==============================] - 1s 2ms/step - loss: 0.4974 - accuracy: 0.8137
Epoch 8/10
455/455 [==============================] - 1s 2ms/step - loss: 0.4960 - accuracy: 0.8140
Epoch 9/10
455/455 [==============================] - 1s 2ms/step - loss: 0.4950 - accuracy: 0.8146
Epoch 10/10
455/455 [==============================] - 1s 2ms/


KeyboardInterrupt



In [ ]:
ALL_DATA

In [ ]:
import json
with open("my_dict.json", "w") as f:
    json.dump(ALL_DATA, f, indent=4)